In [1]:
import ipywidgets as widgets

class MyButton(widgets.Button):
    def __init__(self, name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.description = 'Click me!'
        self.name = name
        self.value = 0
    
    def count(self): self.value += 1

button1 = MyButton('b1')
button2 = MyButton('b2')

def on_button_click(button):
    button.count()
    print(f'Button {button.name} clicked {button.value} times.')

button1.on_click(on_button_click)
button2.on_click(on_button_click)
display(button1)
display(button2)

MyButton(description='Click me!', style=ButtonStyle())

MyButton(description='Click me!', style=ButtonStyle())

### create callback

In [2]:
from time import sleep

def slow_calculation(n=5):
    c = 0
    for i in range(5):
        sleep(1)
        c+=2
    return c

slow_calculation(n=5)

10

In [3]:
def show_progress(c):
    print(f"Epoch {c}")
    
def slow_calculation(cb=None, n=5):
    c = 0
    for i in range(5):
        sleep(1)
        c+=2
        if cb is not None: cb(i)
    return c

slow_calculation(show_progress, n=5)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


10

### Lambdas and partials

In [4]:
# using lambda to create 
slow_calculation(lambda epoch: print(f"Epoch {epoch}!"))

Epoch 0!
Epoch 1!
Epoch 2!
Epoch 3!
Epoch 4!


10

In [5]:
from functools import partial

In [6]:
def show_progress(explaination, epoch):
    print(f"{explaination} Epoch {epoch}.")
    
show_progress("I guess!", 2)

I guess! Epoch 2.


In [7]:
cb = partial(show_progress, "We guess!")

slow_calculation(cb)

We guess! Epoch 0.
We guess! Epoch 1.
We guess! Epoch 2.
We guess! Epoch 3.
We guess! Epoch 4.


10

### Callbacks as callable classes

In [8]:
class CallBack():
    def __init__(self, explaination):
        self.explaination = explaination
        
    def __call__(self, epoch):
        print(f"{self.explaination} Epoch {epoch}.")
        
cb = CallBack("He guesses!")

slow_calculation(cb)

He guesses! Epoch 0.
He guesses! Epoch 1.
He guesses! Epoch 2.
He guesses! Epoch 3.
He guesses! Epoch 4.


10

### Multiple callback funcs; `*args` and `**kwargs`

In [9]:
def my_function(a, *arguments):
    for arg in arguments:
        print(arg)
        
my_function(1, 2, 3) # Output: 1 2 3
my_function('a', 'b', 'c', 'd') # Output: a b c d

2
3
b
c
d


In [10]:
def my_function(**kwargs):
    for k, v in kwargs.items():
        print(k, v)
        
my_function(a=1, b=2, c=3) # Output: 1 2 3

a 1
b 2
c 3


In [11]:
def my_function(*args, **kwargs):
    for v in args:
        print(v)
    for k, v in kwargs.items():
        print(k, v)
        
my_function('a', 'b', 'c', a=1, b=2, c=3) # Output: 1 2 3

a
b
c
a 1
b 2
c 3


In [12]:
# using *args, **kwargs to put general parameters in callbacks

In [13]:
class CallBack():
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
        
    def before_epoch(self, *args, **kwargs): pass
    
    def after_epoch(self, *args, **kwargs): pass
    
def slow_calculation(cb=None, n=5):
    c = 0
    for i in range(5):
        if cb: cb.before_epoch()
        sleep(1)
        c+=2
        if cb: cb.after_epoch()
    return c

cb = CallBack()

slow_calculation(cb)

10

### Using CallBack to modify behavior

In [14]:
class PrintStepCallBack(CallBack):
    def before_epoch(self, epoch, *args, **kwargs): print(f"Before Epoch {epoch}")
    def after_epoch(self, value, *args, **kwargs):
        print(f"After! Value {value}")
        if value > 5: return True
        return False

def slow_calculation(cb=None, n=5):
    c = 0
    for i in range(10):
        if cb: cb.before_epoch(i)
        sleep(1)
        c+=2
        if cb: 
            if cb.after_epoch(c):
                print("Early stopping!")
                break
    return c

cb = PrintStepCallBack()

slow_calculation(cb)

Before Epoch 0
After! Value 2
Before Epoch 1
After! Value 4
Before Epoch 2
After! Value 6
Early stopping!


6

In [15]:
class PrintStepCallBack():
    def after_epoch(self, value, *args, **kwargs):
        print(f"After! Value {value}")
        if value > 5: return True
        return False

def slow_calculation(cb=None, n=5):
    c = 0
    for i in range(10):
        if cb and hasattr(cb, 'before_epoch'): cb.before_epoch(i)
        sleep(1)
        c+=2
        if cb and hasattr(cb, 'after_epoch'): 
            if cb.after_epoch(c):
                print("Early stopping!")
                break
    return c

cb = PrintStepCallBack()

slow_calculation(cb)

After! Value 2
After! Value 4
After! Value 6
Early stopping!


6

### define abitrary cb name to adapt the change in future

In [16]:
class CallBack():
    def callback(self, name, *args, **kwargs):
        cb = getattr(self, name, None)
        if cb: return cb(*args, **kwargs)
        else: return None
    
    def after_epoch(self, value, *args, **kwargs):
        print(f"After! Value {value}")
        if value > 5: return True
        return False

cb = CallBack()


cb.callback('a', 3, c=4)

cb.callback('after_epoch', 3, c=4)

After! Value 3


False

In [17]:
slow_calculation(cb)

After! Value 2
After! Value 4
After! Value 6
Early stopping!


6

In [22]:
class CallBack():
    def callback(self, name, *args, **kwargs):
        cb = getattr(self, name, None)
        if cb: return cb(*args, **kwargs)
        else: return None
    
    def after_epoch(self, value, *args, **kwargs):
        print(f"After! Value {value}")
        if value > 5: return True
        return False

cb = CallBack()

def slow_calculation(cb=None, n=10):
    c = 0
    for i in range(n):
        cb.callback("before_epoch", i)
        sleep(1)
        c+=2
        if cb.callback("after_epoch", c):
            print("Early stopping!")
            break
    return c

slow_calculation(cb)

After! Value 2
After! Value 4
After! Value 6
Early stopping!


6

### inject callback into class to modify value from class

In [40]:
class CallBack():
    def callback(self, name, *args, **kwargs):
        cb = getattr(self, name, None)
        if cb: return cb(*args, **kwargs)
        else: return None
    
    def before_epoch(self, cals, i):
        print(f"Epoch {i}!")
        if cals.value < 4:
            cals.value += 0.1
    
    def after_epoch(self, cals, *args, **kwargs):
        print(f"After! Value {cals.value}")
        if cals.value > 5: return True
        return False
    
class SlowCalculation():
    def __init__(self, cb=None, epoch=10):
        self.cb, self.epoch, self.value = cb, epoch, 0
        
    def callback(self, name, *args, **kwargs):
        if self.cb: cb = getattr(self.cb, name, None)
        if cb: return cb(self, *args, **kwargs)
        else: return None

    def slow_calculation(self):
        for i in range(self.epoch):
            self.callback("before_epoch", i)
            sleep(1)
            self.value += 2
            if self.callback("after_epoch"):
                print("Early stopping!")
                break
    
cb = CallBack()

cals = SlowCalculation(cb)

cals.slow_calculation()

cals.value

Epoch 0!
After! Value 2.1
Epoch 1!
After! Value 4.2
Epoch 2!
After! Value 6.2
Early stopping!


6.2

### `__dunder__` thingies

In [ ]:
# __getitem__
# __getattr__
# __setattr__
# __del__
# __init__
# __new__
# __enter__
# __exit__
# __len__
# __repr__
# __str__

In [77]:
class Basic():
    def __init__(self, xs):
        self.xs = xs
        self.default_value = [2]
    # __getitem__
    def __getitem__(self, i):
        return self.xs[i]
    
    def __getattr__(self, name):
        if name in self.__dict__:
            return self.__dict__[name]
        else:
            return self.default_value
    
    def __setattr__(self, name, value):
        self.__dict__[name] = value
    
    def __repr__(self):
        return 'Basic'
    
    def __str__(self):
        return "Hello!"

X = Basic(list(range(100)))
assert X[0] == 0
assert X[10] == 10
xs = getattr(X, 'xs')
print(xs[0:3])
xs = getattr(X, 'xs3')
print(xs)
X.abc = 2
print(getattr(X, 'abc'))
print(X)
X

[0, 1, 2]
[2]
2
Hello!


Basic